<h1>IBM Data Science Professional Certificate Capstone Project <h1>
<h2>Code for bus tour company expansion -  Date: 12 January 2019 - By Lucas Muller  <h2>
<h3>Step 1: Import libraries<h3>

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

<h3>Step 2: Import GPS neighbourhood locations <h3>

In [2]:
latitude = 43.6532

longitude = -79.3802



In [3]:
Van_df = pd.read_csv('Van_hoods.csv')

Van_df.head()

Van_data=Van_df.drop(['type','scale', 'sym' , 'ge_hotspot'] ,axis=1)

Van_data.head()

,latitude,longitude,name
0,49.246316,-123.163438,Arbutus-Ridge
1,49.279594,-123.115711,Downtown
2,49.238770,-123.187580,Dunbar-Southlands
3,49.263254,-123.130439,Fairview
4,49.274615,-123.065973,Grandview-Woodland


<h3>Step 3: Create Foursquare Call with personal credentials <h3>

In [4]:
CLIENT_ID = '0NQN04U1JVDESO0VPD3JJEHI1LZJ2E2NG4T1O2RBZVYW5WIZ' # your Foursquare ID
CLIENT_SECRET = 'YYQ0VRDGU013CRRRRTGQ0Z3NRKFJK3B3J5SMHJG3OH5KYBA2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0NQN04U1JVDESO0VPD3JJEHI1LZJ2E2NG4T1O2RBZVYW5WIZ
CLIENT_SECRET:YYQ0VRDGU013CRRRRTGQ0Z3NRKFJK3B3J5SMHJG3OH5KYBA2


In [5]:
Van_data.loc[0, 'name']

'Arbutus-Ridge'

In [6]:
name_latitude = Van_data.loc[5, 'latitude'] # neighborhood latitude value
name_longitude = Van_data.loc[5, 'longitude'] # neighborhood longitude value

name_name = Van_data.loc[5, 'name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(name_name, 
                                                               name_latitude, 
                                                               name_longitude))

Latitude and longitude values of Hastings-Sunrise are 49.275940488, -123.03813447899999.


In [7]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 2000 # define radius
section = 'trending'
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    name_latitude, 
    name_longitude, 
    radius,
    section,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=0NQN04U1JVDESO0VPD3JJEHI1LZJ2E2NG4T1O2RBZVYW5WIZ&client_secret=YYQ0VRDGU013CRRRRTGQ0Z3NRKFJK3B3J5SMHJG3OH5KYBA2&v=20180605&ll=49.275940488,-123.03813447899999&radius=2000&section=trending&limit=100'

<h3>Step 4: Import as json<h3>

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3a73eddb04f57d92d0cf6e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Hastings-Sunrise',
  'headerFullLocation': 'Hastings-Sunrise, Vancouver',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 120,
  'suggestedBounds': {'ne': {'lat': 49.29394050600002,
    'lng': -123.01059623557836},
   'sw': {'lat': 49.25794046999999, 'lng': -123.06567272242162}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50ada0e0e4b03c6b1ca797ff',
       'name': 'Curry Zone',
       'location': {'lat': 49.28123747726763,
        'lng': -123.05016935156273,
        'labeledLatLngs': [{'label': 'display',
          'lat': 49.28123747726763,
         

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

<h3>Step 5: See what first group looks like<h3>

In [10]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

,name,categories,lat,lng
0,Curry Zone,Indian Restaurant,49.281237,-123.050169
1,The Gray Olive Cafeteria,Café,49.280955,-123.011700
2,Yama Cafe,Café,49.281283,-123.063315
3,Church's Chicken,Fried Chicken Joint,49.281028,-123.057543
4,Admiral Pub & Grill,Pub,49.281188,-123.013607
5,Basil Garden,Vietnamese Restaurant,49.262189,-123.044652
6,Song Huong,Vietnamese Restaurant,49.270146,-123.056783
7,Bao Chau,Vietnamese Restaurant,49.281232,-123.048971
8,Playland,Theme Park,49.281924,-123.036258
9,Campagnolo Roma,Italian Restaurant,49.281343,-123.057953


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


<h3>Step 6: Create loop to get near by trending venues for each neighbourhood <h3>

In [12]:
# LIMIT = 100 # limit of number of venues returned by Foursquare API

# radius = 500 # define radius

# section = 'food'

def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
       
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&section={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            section,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['name', 
                  'name Latitude', 
                  'name Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
Van_venues = getNearbyVenues(names=Van_data['name'],
                                   latitudes=Van_data['latitude'],
                                   longitudes=Van_data['longitude']
                                  )

Arbutus-Ridge
Downtown
Dunbar-Southlands
Fairview
Grandview-Woodland
Hastings-Sunrise
Kensington-Cedar Cottage
Kerrisdale
Killarney
Kitsilano
Marpole
Mount Pleasant
Oakridge
Renfrew-Collingwood
Riley Park
Shaughnessy
South Cambie
Strathcona
Sunset
Victoria-Fraserview
West End
West Point Grey


In [14]:
print(Van_venues.shape)
Van_venues.head()

(1983, 7)


,name,name Latitude,name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arbutus-Ridge,49.246316,-123.163438,Asa Sushi,49.234615,-123.157935,Sushi Restaurant
1,Arbutus-Ridge,49.246316,-123.163438,Ridge Gardens Restaurant,49.258073,-123.153394,Chinese Restaurant
2,Arbutus-Ridge,49.246316,-123.163438,Crépe Café,49.234683,-123.182066,Café
3,Arbutus-Ridge,49.246316,-123.163438,Modern City Chinese Restaurant,49.234590,-123.155961,Chinese Restaurant
4,Arbutus-Ridge,49.246316,-123.163438,Bean Brothers,49.234713,-123.157260,Sandwich Place


In [15]:
Van_venues.groupby('name').count()

,name Latitude,name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
name,,,,,,
Arbutus-Ridge,100,100,100,100,100,100
Downtown,100,100,100,100,100,100
Dunbar-Southlands,40,40,40,40,40,40
Fairview,100,100,100,100,100,100
Grandview-Woodland,100,100,100,100,100,100
Hastings-Sunrise,100,100,100,100,100,100
Kensington-Cedar Cottage,100,100,100,100,100,100
Kerrisdale,65,65,65,65,65,65
Killarney,58,58,58,58,58,58


In [16]:
print('There are {} uniques categories.'.format(len(Van_venues['Venue Category'].unique())))

There are 197 uniques categories.


<h3>Step 7: one hot encoding <h3>

In [17]:
# one hot encoding
Van_onehot = pd.get_dummies(Van_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Van_onehot['name'] = Van_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [Van_onehot.columns[-1]] + list(Van_onehot.columns[:-1])
Van_onehot = Van_onehot[fixed_columns]

Van_onehot.head(5)

,name,Accessories Store,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Arbutus-Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [18]:
Van_onehot.shape

(1983, 198)

In [19]:
Van_grouped = Van_onehot.groupby('name').mean().reset_index()
Van_grouped

,name,Accessories Store,American Restaurant,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hobby Shop,Hockey Arena,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Smoothie Shop,Soccer Field,Soccer Stadium,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Shop,Women's Store,Yoga Studio
0,Arbutus-Ridge,0.00,0.000000,0.00,0.00,0.00,0.010000,0.00,0.00,0.000000,0.000000,0.030000,0.020000,0.000000,0.01,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.020000,0.00,0.000000,0.000000,0.010,0.00,0.00,0.040000,0.00,0.00,0.00,0.00,0.130000,0.00,0.000000,0.00,0.00,0.010000,0.010000,0.00,0.00,0.01,0.00,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.00,0.010000,0.000000,0.000000,0.00,0.010000,0.030000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.050000,0.03,0.020000,0.010000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.010000,0.010000,0.010,0.00,0.000000,0.020000,0.030000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.01,0.00,0.00,0.00,0.00,0.010000,0.00,0.010000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.010000,0.000000,0.00,0.00,0.00,0.00,0.040000,0.000000,0.010000,0.020000,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.010000,0.010000,0.00,0.00,0.00,0.000000,0.010000,0.00,0.010000,0.060000,0.00,0.000000,0.

In [20]:
Van_grouped.shape

(22, 198)

<h3>Step 8: create a loop to group the venues based on frequncy for each neighbourhood <h3>

In [21]:
num_top_venues = 5

for hood in Van_grouped['name']:
    print("----"+hood+"----")
    temp = Van_grouped[Van_grouped['name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arbutus-Ridge----
              venue  freq
0       Coffee Shop  0.13
1    Sandwich Place  0.06
2     Grocery Store  0.05
3  Sushi Restaurant  0.04
4              Park  0.04


----Downtown----
                 venue  freq
0          Coffee Shop  0.07
1   Seafood Restaurant  0.05
2  Japanese Restaurant  0.05
3           Restaurant  0.05
4       Sandwich Place  0.04


----Dunbar-Southlands----
                 venue  freq
0     Sushi Restaurant  0.08
1          Coffee Shop  0.08
2          Golf Course  0.08
3  Japanese Restaurant  0.05
4   Italian Restaurant  0.05


----Fairview----
                 venue  freq
0  Japanese Restaurant  0.07
1           Restaurant  0.05
2               Bakery  0.04
3                 Café  0.04
4     Sushi Restaurant  0.04


----Grandview-Woodland----
              venue  freq
0       Coffee Shop  0.10
1           Brewery  0.08
2              Café  0.05
3       Pizza Place  0.05
4  Sushi Restaurant  0.05


----Hastings-Sunrise----
                   ven

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
name_venues_sorted = pd.DataFrame(columns=columns)
name_venues_sorted['name'] = Van_grouped['name']

for ind in np.arange(Van_grouped.shape[0]):
    name_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Van_grouped.iloc[ind, :], num_top_venues)

name_venues_sorted.head()

,name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Arbutus-Ridge,Coffee Shop,Sandwich Place,Grocery Store,Sushi Restaurant,Chinese Restaurant
1,Downtown,Coffee Shop,Japanese Restaurant,Restaurant,Seafood Restaurant,Sandwich Place
2,Dunbar-Southlands,Golf Course,Sushi Restaurant,Coffee Shop,Bakery,Park
3,Fairview,Japanese Restaurant,Restaurant,Coffee Shop,Bakery,Café
4,Grandview-Woodland,Coffee Shop,Brewery,Pizza Place,Sushi Restaurant,Café


<h3>Step 9: Use K-Means Clustering model to group the neighbourhoods based on similarities in venues<h3>

In [24]:
# set number of clusters
kclusters = 5

Van_grouped_clustering = Van_grouped.drop('name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Van_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:] 

array([1, 3, 0, 4, 3, 3, 2, 2, 2, 4, 1, 3, 1, 2, 1, 1, 1, 3, 2, 2, 4, 4],
      dtype=int32)

In [25]:
Van_data.head()

,latitude,longitude,name
0,49.246316,-123.163438,Arbutus-Ridge
1,49.279594,-123.115711,Downtown
2,49.238770,-123.187580,Dunbar-Southlands
3,49.263254,-123.130439,Fairview
4,49.274615,-123.065973,Grandview-Woodland


In [26]:
Van_merged = Van_data

# add clustering labels
Van_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Van_merged = Van_merged.join(name_venues_sorted.set_index('name'), on='name')

Van_merged.head(30) # check the last columns!

,latitude,longitude,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,49.246316,-123.163438,Arbutus-Ridge,1,Coffee Shop,Sandwich Place,Grocery Store,Sushi Restaurant,Chinese Restaurant
1,49.279594,-123.115711,Downtown,3,Coffee Shop,Japanese Restaurant,Restaurant,Seafood Restaurant,Sandwich Place
2,49.238770,-123.187580,Dunbar-Southlands,0,Golf Course,Sushi Restaurant,Coffee Shop,Bakery,Park
3,49.263254,-123.130439,Fairview,4,Japanese Restaurant,Restaurant,Coffee Shop,Bakery,Café
4,49.274615,-123.065973,Grandview-Woodland,3,Coffee Shop,Brewery,Pizza Place,Sushi Restaurant,Café
5,49.275940,-123.038134,Hastings-Sunrise,3,Vietnamese Restaurant,Park,Coffee Shop,Café,Grocery Store
6,49.246839,-123.070497,Kensington-Cedar Cottage,2,Vietnamese Restaurant,Chinese Restaurant,Coffee Shop,Sandwich Place,Grocery Store
7,49.224024,-123.159310,Kerrisdale,2,Coffee Shop,Chinese Restaurant,Sushi Restaurant,Golf Course,Sandwich Place
8,49.218820,-123.036324,Killarney,2,Park,Coffee Shop,Chinese Restaurant,Golf Course,Sandwich Place
9,49.265832,-123.164366,Kitsilano,4,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Park,Restaurant


<h3>Step 10: Visualize clusters on a folium map <h3>

In [27]:
latitude =  49.246292

longitude = -123.116226


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Van_merged['latitude'], Van_merged['longitude'], Van_merged['name'], Van_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Step 11: Explore the clusters <h3>

In [28]:

Van_merged.loc[Van_merged['Cluster Labels'] ==1 ]

,latitude,longitude,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,49.246316,-123.163438,Arbutus-Ridge,1,Coffee Shop,Sandwich Place,Grocery Store,Sushi Restaurant,Chinese Restaurant
10,49.210017,-123.129178,Marpole,1,Clothing Store,Coffee Shop,Chinese Restaurant,Sporting Goods Shop,Vietnamese Restaurant
12,49.225099,-123.121965,Oakridge,1,Coffee Shop,Fast Food Restaurant,Garden,Sandwich Place,Sushi Restaurant
14,49.244954,-123.101517,Riley Park,1,Coffee Shop,Café,Vietnamese Restaurant,Chinese Restaurant,Grocery Store
15,49.246118,-123.138868,Shaughnessy,1,Coffee Shop,Chinese Restaurant,Sandwich Place,Furniture / Home Store,Breakfast Spot
16,49.245830,-123.122336,South Cambie,1,Coffee Shop,Café,Chinese Restaurant,Breakfast Spot,Sushi Restaurant


In [29]:
Van_merged.loc[Van_merged['Cluster Labels'] ==2 ]

,latitude,longitude,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,49.246839,-123.070497,Kensington-Cedar Cottage,2,Vietnamese Restaurant,Chinese Restaurant,Coffee Shop,Sandwich Place,Grocery Store
7,49.224024,-123.159310,Kerrisdale,2,Coffee Shop,Chinese Restaurant,Sushi Restaurant,Golf Course,Sandwich Place
8,49.218820,-123.036324,Killarney,2,Park,Coffee Shop,Chinese Restaurant,Golf Course,Sandwich Place
13,49.248189,-123.038794,Renfrew-Collingwood,2,Chinese Restaurant,Pizza Place,Vietnamese Restaurant,Coffee Shop,Convenience Store
18,49.219301,-123.091509,Sunset,2,Chinese Restaurant,Sandwich Place,Coffee Shop,Japanese Restaurant,Park
19,49.220438,-123.063745,Victoria-Fraserview,2,Chinese Restaurant,Japanese Restaurant,Sandwich Place,Asian Restaurant,Park


In [30]:
Van_merged.loc[Van_merged['Cluster Labels'] ==3 ]

,latitude,longitude,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,49.279594,-123.115711,Downtown,3,Coffee Shop,Japanese Restaurant,Restaurant,Seafood Restaurant,Sandwich Place
4,49.274615,-123.065973,Grandview-Woodland,3,Coffee Shop,Brewery,Pizza Place,Sushi Restaurant,Café
5,49.275940,-123.038134,Hastings-Sunrise,3,Vietnamese Restaurant,Park,Coffee Shop,Café,Grocery Store
11,49.262392,-123.101199,Mount Pleasant,3,Coffee Shop,Brewery,Café,Japanese Restaurant,Vietnamese Restaurant
17,49.275330,-123.088607,Strathcona,3,Brewery,Coffee Shop,Pizza Place,Sandwich Place,Sushi Restaurant


In [31]:
Van_merged.loc[Van_merged['Cluster Labels'] ==4]

,latitude,longitude,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,49.263254,-123.130439,Fairview,4,Japanese Restaurant,Restaurant,Coffee Shop,Bakery,Café
9,49.265832,-123.164366,Kitsilano,4,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Park,Restaurant
20,49.286084,-123.134352,West End,4,Café,Restaurant,Japanese Restaurant,Coffee Shop,Seafood Restaurant
21,49.266813,-123.203267,West Point Grey,4,Sushi Restaurant,Coffee Shop,Japanese Restaurant,Park,Fast Food Restaurant


In [32]:
Van_merged.loc[Van_merged['Cluster Labels'] ==0]

,latitude,longitude,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,49.23877,-123.18758,Dunbar-Southlands,0,Golf Course,Sushi Restaurant,Coffee Shop,Bakery,Park
